In [1]:
import os
import math
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from astroquery import sha
# from astropy import coordinates as coord
# from astropy import units as u
# import requests
# import pyvo as vo

PI = math.pi

In [2]:
#TODO: ESTABLISH DICTIONARY OF PLANET NAMES
#      CENTRALIZE TABLES


systems_df = pd.read_csv('data/PS_2021.11.10_14.15.35.csv', sep=',', low_memory=False)
transitspec_df = pd.read_csv('data/transitspec_2021.10.21_17.04.26.csv', sep=',', low_memory=False)
emispec_df = pd.read_csv('data/emissionspec_2021.10.22_13.36.00.csv', sep=',', low_memory=False)

In [3]:
#what are the MOST ESSENTIAL features?
for col in systems_df.columns:
    print(col)

pl_name
hostname
default_flag
sy_snum
sy_pnum
pl_controv_flag
pl_orbper
pl_orbsmax
pl_rade
pl_radj
pl_bmasse
pl_bmassj
pl_bmassprov
pl_orbeccen
pl_insol
pl_eqt
ttv_flag
st_spectype
st_teff
st_rad
st_mass
st_met
st_metratio
st_logg
rastr
ra
decstr
dec
sy_dist
sy_vmag
sy_kmag
sy_gaiamag


In [15]:
names = systems_df['pl_name']
earth_masses = systems_df['pl_bmasse']
num_stars = systems_df['sy_snum']
num_planets = systems_df['sy_pnum']

#remove rows with nonzero 'contraversial' flag
systems_df = systems_df[systems_df['pl_controv_flag'] == 0]

#filter only for readings that are considered the 'default' reading for a particular planet
systems_defaultonly_df = systems_df[systems_df['default_flag'] == 1]

#the number of non-contraversial planets in the table
print(len(systems_defaultonly_df))

#extract the stellar features of highest importance from this table
# - Host Star Name (optional)
# - Stellar Mass
# - Stellar Metallicity
# - Stellar Effective temperature
starinfo_df = systems_df[['st_mass', 'st_met', 'st_teff']]

#define these same features for our sun (source: Wikipedia)
SUN_MASS = 1
SUN_MET = 0.0122
SUN_TEFF = 5772

m_sun = np.array([SUN_MASS,
                 SUN_MET,
                 SUN_TEFF])

4542


In [16]:
starinfo_x = starinfo_df.dropna()

starinfo_mean = starinfo_x.mean();

print("mean vector of stellar features:\n {0} \n".format(starinfo_mean))

print("covariance of key stellar features")
cov_X = starinfo_x.cov()
print(cov_X)

print("\ncorrelation of key stellar features")
corr_X = starinfo_x.corr()
print(corr_X)

print(starinfo_x)

mean vector of stellar features:
 st_mass       0.963049
st_met       -0.032020
st_teff    5506.199830
dtype: float64 

covariance of key stellar features
            st_mass    st_met        st_teff
st_mass    0.108597  0.015115     130.920613
st_met     0.015115  0.051251       8.992934
st_teff  130.920613  8.992934  512870.653409

correlation of key stellar features
          st_mass    st_met   st_teff
st_mass  1.000000  0.202608  0.554746
st_met   0.202608  1.000000  0.055469
st_teff  0.554746  0.055469  1.000000
       st_mass  st_met  st_teff
0         2.70  -0.350  4742.00
3         2.78  -0.020  4213.00
4         1.80   0.040  4340.00
6         2.20  -0.240  4813.00
7         0.97   0.405  5314.94
...        ...     ...      ...
29975     1.15   0.101  6105.51
29978     1.15   0.101  6105.51
29981     1.29   0.122  6156.77
29982     1.31   0.131  6089.00
29984     2.20  -0.205  4780.00

[18453 rows x 3 columns]


In [29]:
#####################################################
#DEFINE INPUT SPACE
#####################################################

X = starinfo_x.to_numpy()
print (X.shape)
print(X)


(18453, 3)
[[ 2.70000e+00 -3.50000e-01  4.74200e+03]
 [ 2.78000e+00 -2.00000e-02  4.21300e+03]
 [ 1.80000e+00  4.00000e-02  4.34000e+03]
 ...
 [ 1.29000e+00  1.22000e-01  6.15677e+03]
 [ 1.31000e+00  1.31000e-01  6.08900e+03]
 [ 2.20000e+00 -2.05000e-01  4.78000e+03]]


In [31]:
#PCA
from sklearn.decomposition import KernelPCA, FactorAnalysis
pca = KernelPCA(n_components=2, kernel='linear')
pca.fit(X)

print(f"Eigenvectors from PCA: {pca.eigenvectors_}\n")
print(f"Eigenvalues from PCA: {pca.eigenvalues_}\n")

Eigenvectors from PCA: [[-0.00785563  0.04281078]
 [-0.01329351  0.05141511]
 [-0.01198801  0.02793375]
 ...
 [ 0.00668757  0.00529306]
 [ 0.00599093  0.00628489]
 [-0.00746501  0.03217487]]

Eigenvalues from PCA: [9.46348992e+09 1.48951042e+03]



In [ ]:
#we need to extract classes or habitability metric in a deterministic fashion from this data.
#Do this using multivariate Gaussian distribution centered around sun data.
                                                                   
#gaussian: (mean vector, cov matrix) -> (num -> prob)
def gaussian(mu, sig):
    return lambda x : (1 / (2 * PI * np.linalg.det(sig))**(1/2)) * np.exp(-1/2 * np.transpose(np.subtract(x, mu)) * np.linalg.inv(sig) * np.subtract(x,mu))


sun_likeness = gaussian(m_sun, corr_X)

sun_likeness(np.array(starinfo_x)[:, 1:])


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

In [ ]:

#The above mean, covariance can parameterize a multivariate gaussian distribution

#Each bayesian update will take inner producc of star vector and sun  data

#Gaussian distribution is updated by each entry in star table



